![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [148]:
import pandas as pd
import numpy as np

# Start coding here...

In [149]:
# Importing data using pandas
bank_marketing = pd.read_csv('bank_marketing.csv')
bank_marketing.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [150]:
# Splitting the main Dataframe into client, campaign and economics

client = bank_marketing[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]

# Added all the columns except last_contact_date which has to be created
campaign = bank_marketing[["client_id", "number_contacts", "month", "day", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]

economics = bank_marketing[["client_id", "cons_price_idx", "euribor_three_months"]]

In [151]:
# Verifying the columns
print(client.columns)
print(campaign.columns)
print(economics.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage'],
      dtype='object')
Index(['client_id', 'number_contacts', 'month', 'day', 'contact_duration',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome'],
      dtype='object')
Index(['client_id', 'cons_price_idx', 'euribor_three_months'], dtype='object')


In [152]:
# Comparing the data types with the types required
print(client.dtypes)
print(campaign.dtypes)
print(economics.dtypes)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default    object
mortgage          object
dtype: object
client_id                      int64
number_contacts                int64
month                         object
day                            int64
contact_duration               int64
previous_campaign_contacts     int64
previous_outcome              object
campaign_outcome              object
dtype: object
client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object


In [153]:
## Editing the client dataset
# Clean job column
client["job"] = client["job"].str.replace(".", "_")

# Clean education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# Clean and convert credit_default, mortgage to bool data type
for col in ["credit_default", "mortgage"]:
    client[col] = client[col].map({"yes" : 1, "no": 0, "unknown": 0})
    client[col] = client[col].astype(bool)

## Editing the campaign dataset
# Change previous_outcome to binary values
print(campaign["previous_outcome"].unique())
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "nonexistent": 0, "failure": 0})

# Change campaign_outcome to binary values
print(campaign["campaign_outcome"].unique())
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Create last_contact_data column
# Add year column
campaign["year"] = "2022"


['nonexistent' 'failure' 'success']
['no' 'yes']
